In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..')))
from src.interaction.message import ChatMessageGenerator
from src.credential_manager.LocalCredentials import LocalCredentials
from src.model.model_catalogue import ModelType, EmbeddingType, Providers, ModelCatalogue
from src.model.wrappers import ChatModelWrapper, EmbeddingWrapper
from src.data.attachments import AttachmentTypes, Attachment


c:\Users\Student\Documents\University\Fifth Year\COM4520\Psycore\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = ModelCatalogue._models["grok_2_vision"]
model_wrapper = ChatModelWrapper(model)
attach = [Attachment(AttachmentTypes.IMAGE, "dog.jpeg", True),Attachment(AttachmentTypes.IMAGE, "cat.jpeg", True)]
Attachment.extractAttachmentList(attach)
print(model.multiModal)

generator = ChatMessageGenerator(model_wrapper,"Describe all user prompts")
prepped_message = generator.prepResponse(attachments=attach)
print(prepped_message)

True
('user', [{'type': 'text', 'text': '{prompt}'}, {'type': 'image_url', 'image_url': {'url': 'data:image/jpeg;base64,{attachment0}'}}, {'type': 'image_url', 'image_url': {'url': 'data:image/jpeg;base64,{attachment1}'}}])


In [3]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

generator.chat.append(prepped_message)
template = ChatPromptTemplate(generator.chat)
chain = template | model_wrapper.model

input_variables = Attachment.attachmentListMapping(attach)
print(input_variables)
input_variables.update({"prompt": "What's in the images."})
response = chain.invoke(input_variables)
print(response)


{'attachment0': '', 'attachment1': ''}content='Here are the descriptions of the user prompts for the images provided:\n\n1. **First Image:**\n   - **Prompt:** "What\'s in the image?"\n   - **Description:** The image shows a cute, fluffy golden retriever puppy sitting on a white background. The puppy has a light golden coat and is wearing an orange collar with a silver buckle.\n\n2. **Second Image:**\n   - **Prompt:** "What\'s in the image?"\n   - **Description:** The image features an adorable kitten sitting on a wooden floor. The kitten has a white and gray tabby coat with large, expressive eyes. The background is blurred, focusing attention on the kitten.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 134, 'prompt_tokens': 533, 'total_tokens': 667, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0, 'text_tokens': 21, 'image_tokens': 512}}, 'model_name': 'grok-2-vision-1212', 'system_fingerp

In [4]:
from src.interaction.knowledge_graphs import ModelKGTransformer
kg_transformer = ModelKGTransformer(ModelCatalogue._models["grok_2_vision"])
